<a href="https://colab.research.google.com/github/germanEvarela/pandas/blob/main/ScrapingBeautifulSoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bs4

In [ ]:
!pip install Beautifulsoup4

In [49]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [50]:
url_inicial = 'https://books.toscrape.com/index.html'
url_root = 'https://books.toscrape.com/index.html'

In [51]:
r = requests.get(url_inicial)

In [52]:
r.status_code

200

In [53]:
s = BeautifulSoup(r.text,'lxml')


In [ ]:
lista_article = s.find_all('article', class_ ='product_pod')
links_libros = [x.find('h3').find('a').get('href') for x in lista_article]
links_libros = [urljoin(url_root, i) for i in links_libros]
links_libros

In [55]:
# funcion de obtener links
def get_url_items(sopa, url):
    url_root = 'https://books.toscrape.com/index.html'
    lista_article = s.find_all('article', class_ ='product_pod')
    links_libros = [x.find('h3').find('a').get('href') for x in lista_article]
    links_libros = [urljoin(url_root, i) for i in links_libros]
    return links_libros



In [56]:
links_items = []
i = 0
while i<50:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag = requests.get(url_inicial)
    s_pag = BeautifulSoup(r_pag.text,'lxml')
    links = get_url_items(s_pag, url_inicial)
    next_a = s_pag.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))
    links_items.append(links)


Estoy en la pagina https://books.toscrape.com/index.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-2.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-3.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-4.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-5.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-6.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-7.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-8.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-9.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-10.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-11.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-12.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-13.html
Estoy en la pagina https://books.toscrape.com/catalogue/page-14.html
Estoy en la pagina https://books.toscrape.com/catalogu

In [ ]:
links_items

In [58]:
len(links_items)

49

In [59]:
list_scraper = []
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

980

In [60]:
uno = list_scraper[0]
r_item = requests.get(uno)
s_item = BeautifulSoup(r_item.text, 'lxml')

In [76]:
from IPython.display import Image

In [ ]:
# titulo del libro
titulo = s_item.find('h1').get_text(strip=True)

# obtener precio
precio = s_item.find('p', class_='price_color').get_text(strip=True)

# obtener descripcion
#ancla_desc = s_item.find('div', id='Product Description')
#ancla_desc.find_next_sibling('p').get_text(strip=True)

# obtener imagen
src_img = s_item.find('div', class_='item active').find('img').get('src')
src_img = urljoin(url_root, src_img)
Image(requests.get(src_img).content)



In [80]:
# funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book = {}
    r = requests.get(url)
    s_item = BeautifulSoup(r.text, 'lxml')
    titulo = s_item.find('h1').get_text(strip=True)
    if titulo:
        content_book['Titulo'] = titulo
    else:
        content_book['Titulo'] = None
    precio = s_item.find('p', class_='price_color').get_text(strip=True)
    if precio:
        content_book['Precio'] = precio
    else:
        content_book['Precio'] = None
    src_img = s_item.find('div', class_='item active').find('img').get('src')
    if src_img:
        content_book['Url_img'] = urljoin(url_root, src_img)
    else:
        content_book['Url_img'] = None
    return content_book



In [81]:
list_scraper = list_scraper[0:10]
datos_book = []
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9


In [ ]:
datos_book

In [83]:
df_catalogo = pd.DataFrame(datos_book)
df_catalogo

,Titulo,Precio,Url_img
0,A Light in the Attic,Â£51.77,https://books.toscrape.com/media/cache/fe/72/f...
1,Tipping the Velvet,Â£53.74,https://books.toscrape.com/media/cache/08/e9/0...
2,Soumission,Â£50.10,https://books.toscrape.com/media/cache/ee/cf/e...
3,Sharp Objects,Â£47.82,https://books.toscrape.com/media/cache/c0/59/c...
4,Sapiens: A Brief History of Humankind,Â£54.23,https://books.toscrape.com/media/cache/ce/5f/c...
5,The Requiem Red,Â£22.65,https://books.toscrape.com/media/cache/6b/07/6...
6,The Dirty Little Secrets of Getting Your Dream...,Â£33.34,https://books.toscrape.com/media/cache/e1/1b/e...
7,The Coming Woman: A Novel Based on the Life of...,Â£17.93,https://books.toscrape.com/media/cache/97/36/9...
8,The Boys in the Boat: Nine Americans and Their...,Â£22.60,https://books.toscrape.com/media/cache/d1/2d/d...
9,The Black Maria,Â£52.15,https://books.toscrape.com/media/cache/d1/7a/d...


In [88]:
from IPython.core.display import HTML

def path_html_img(url):
    return '<img src = "'+ url+'"width="60">'

In [89]:
df_catalogo['Vis_img']=df_catalogo['Url_img'].apply(lambda x : path_html_img(x))
df_catalogo

,Titulo,Precio,Url_img,Vis_img
0,A Light in the Attic,Â£51.77,https://books.toscrape.com/media/cache/fe/72/f...,"<img src = ""https://books.toscrape.com/media/c..."
1,Tipping the Velvet,Â£53.74,https://books.toscrape.com/media/cache/08/e9/0...,"<img src = ""https://books.toscrape.com/media/c..."
2,Soumission,Â£50.10,https://books.toscrape.com/media/cache/ee/cf/e...,"<img src = ""https://books.toscrape.com/media/c..."
3,Sharp Objects,Â£47.82,https://books.toscrape.com/media/cache/c0/59/c...,"<img src = ""https://books.toscrape.com/media/c..."
4,Sapiens: A Brief History of Humankind,Â£54.23,https://books.toscrape.com/media/cache/ce/5f/c...,"<img src = ""https://books.toscrape.com/media/c..."
5,The Requiem Red,Â£22.65,https://books.toscrape.com/media/cache/6b/07/6...,"<img src = ""https://books.toscrape.com/media/c..."
6,The Dirty Little Secrets of Getting Your Dream...,Â£33.34,https://books.toscrape.com/media/cache/e1/1b/e...,"<img src = ""https://books.toscrape.com/media/c..."
7,The Coming Woman: A Novel Based on the Life of...,Â£17.93,https://books.toscrape.com/media/cache/97/36/9...,"<img src = ""https://books.toscrape.com/media/c..."
8,The Boys in the Boat: Nine Americans and Their...,Â£22.60,https://books.toscrape.com/media/cache/d1/2d/d...,"<img src = ""https://books.toscrape.com/media/c..."
9,The Black Maria,Â£52.15,https://books.toscrape.com/media/cache/d1/7a/d...,"<img src = ""https://books.toscrape.com/media/c..."


In [90]:
HTML(df_catalogo.to_html(escape=False, formatters=dict(Portada=path_html_img)))

,Titulo,Precio,Url_img,Vis_img
0,A Light in the Attic,Â£51.77,https://books.toscrape.com/media/cache/fe/72/fe72f0532301ec28892ae79a629a293c.jpg,
1,Tipping the Velvet,Â£53.74,https://books.toscrape.com/media/cache/08/e9/08e94f3731d7d6b760dfbfbc02ca5c62.jpg,
2,Soumission,Â£50.10,https://books.toscrape.com/media/cache/ee/cf/eecfe998905e455df12064dba399c075.jpg,
3,Sharp Objects,Â£47.82,https://books.toscrape.com/media/cache/c0/59/c05972805aa7201171b8fc71a5b00292.jpg,
4,Sapiens: A Brief History of Humankind,Â£54.23,https://books.toscrape.com/media/cache/ce/5f/ce5f052c65cc963cf4422be096e915c9.jpg,
5,The Requiem Red,Â£22.65,https://books.toscrape.com/media/cache/6b/07/6b07b77236b7c80f42bd90bf325e69f6.jpg,
6,The Dirty Little Secrets of Getting Your Dream Job,Â£33.34,https://books.toscrape.com/media/cache/e1/1b/e11bea016d0ae1d7e2dd46fb3cb870b7.jpg,
7,"The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull",Â£17.93,https://books.toscrape.com/media/cache/97/36/9736132a43b8e6e3989932218ef309ed.jpg,
8,The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics,Â£22.60,https://books.toscrape.com/media/cache/d1/2d/d12d26739b5369a6b5b3024e4d08f907.jpg,
9,The Black Maria,Â£52.15,https://books.toscrape.com/media/cache/d1/7a/d17a3e313e52e1be5651719e4fba1d16.jpg,
